<a href="https://colab.research.google.com/github/TheBlackRus/liveproject_HumanPoseEstimation/blob/master/liveproject_humanpose_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to part 2. Training of a CNN wiht pytorch

In [0]:
#!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
#!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat


In [0]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from scipy.io import loadmat
from matplotlib.pyplot import imshow

In [0]:
from torch.utils.data import Dataset
class SVHN_dataset(Dataset):

    def __init__(self, data):
        # Transform data to Torch Tensors
        self.images = torch.tensor(data['X']).permute([3,2,0,1])
        self.labels = torch.tensor(data['y'])
        self.size = self.labels.shape[0]

        # replace label 10 with label 0
        self.labels[np.where(self.labels == 10)] = 0

        # convert to float and normalize images to 0..1 range
        self.images = self.images.to(dtype=torch.float)
        self.images = self.images / 255.0

        self.labels = self.labels.to(dtype=torch.long)


    
    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [0]:
train_data = loadmat('train_32x32.mat')
test_data = loadmat('test_32x32.mat')

In [0]:
train_dataset = SVHN_dataset(train_data)
test_dataset = SVHN_dataset(test_data)

Values are supposed to be normalized

In [5]:
print("images range [", train_dataset.images.min(),",", train_dataset.images.max(),"]",train_dataset.images.shape)
print("labels",train_dataset.labels.unique())

images range [ tensor(0.) , tensor(1.) ] torch.Size([73257, 3, 32, 32])
labels tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


Creating the net
1. convolution, kernel_size=5, channels=6, stride=1, padding=2
2. batch-normalization
3. ReLU
4. Max-pool, kernel_size=2, stride=2

5. convolution, kernel_size=3, channels=12, stride=1, padding=1
6. batch-normalization
7. ReLU
8. Max-pool, kernel_size=2, stride=2

9. convolution, kernel_size=3, channels=24, stride=1, padding=1
10. batch-normalization
11. ReLU
12. Max-pool, kernel_size=2, stride=2

13. fully connected layer, output_size=10

In [0]:
import torch.nn as nn
import torch.nn.functional as F


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #block
        self.conv_1 = nn.Conv2d(in_channels = 3, out_channels=6,kernel_size=5,stride=1,padding=2)
        #nn.init.normal_(self.conv_1.weight, std=0.001)
        #nn.init.constant_(self.conv_1.bias,0)
        self.batch_norm_1 = nn.BatchNorm2d(6) # same as out_channels before layer 
        self.relu_1 = nn.ReLU()
        self.pool_1 = nn.MaxPool2d(2, 2)


        self.conv_2 = nn.Conv2d(in_channels = 6, out_channels=12,kernel_size=3,stride=1,padding=1)
        #nn.init.normal_(self.conv_2.weight, std=0.001)
        #nn.init.constant_(self.conv_2.bias,0)
        self.batch_norm_2 = nn.BatchNorm2d(12) # same as out_channels before layer 
        self.relu_2 = nn.ReLU()
        self.pool_2 = nn.MaxPool2d(2, 2)

        self.conv_3 = nn.Conv2d(in_channels= 12, out_channels=24,kernel_size=3,stride=1,padding=1)
        #nn.init.normal_(self.conv_3.weight, std=0.001)
        #nn.init.constant_(self.conv_3.bias,0)
        self.batch_norm_3 = nn.BatchNorm2d(24) # same as out_channels before layer 
        self.relu_3 = nn.ReLU()
        self.pool_3 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(24 * 4 * 4, 10)# 24 chans x 32//(2*2*2)
        #nn.init.normal_(self.fc1.weight, std=0.001)
        #nn.init.constant_(self.fc1.bias,0)

    def forward(self, x):
        x = self.conv_1(x)
        x = self.batch_norm_1(x)
        x = self.relu_1(x)
        x = self.pool_1(x)

        x = self.conv_2(x)
        x = self.batch_norm_2(x)
        x = self.relu_2(x)
        x = self.pool_2(x)

        x = self.conv_3(x)
        x = self.batch_norm_3(x)
        x = self.relu_3(x)
        x = self.pool_3(x)

        x = x.view(-1, 24 * 4 * 4)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc1(x)
        return x



net = Net()


In [8]:
# import torchvision
"""
trainset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
"""


"\ntrainset = torchvision.datasets.CIFAR10(root='./data', train=False,\n                                        download=True)\ntrainloader = torch.utils.data.DataLoader(trainset, batch_size=4,\n                                          shuffle=True, num_workers=2)\n"

Define Loss

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.01) # ,lr=0.001 #optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=256,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=256,
                                          shuffle=False, num_workers=2)

"""
trainloader.dataset.labels
"""


'\ntrainloader.dataset.labels\n'

In [0]:
def get_accuracy(model, dataloader):

    model.eval()
    # TODO: calculate accuracy
    acc = 0
    count = 0
    for i, data in enumerate(dataloader, 0):
      inputs, labels = data
      labels = labels.squeeze()
      inputs = inputs.cuda()
      labels = labels.cuda()
      
      outputs = net(inputs)
      predicted = torch.argmax(outputs, dim=1)
      #print(predicted, labels)
      acc = acc + (predicted == labels).sum()
      count = count + labels.shape[0]
    #print("AC",acc,count)
    acc = acc.to(torch.float)
    
    if count >0:
      accuracy = acc/count
    else:
      accuracy = 0
    #outputs = net(inputs)
    model.train()
    
    return accuracy

In [11]:
print("device","cuda" if torch.cuda.is_available() else "cpu")
net.cuda()
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels = labels.squeeze()
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        outputs.cuda()
        #print(labels,labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            acc = get_accuracy(net,testloader)
            print("acc" , acc)
            running_loss = 0.0

print('Finished Training')

device cuda
[1,   100] loss: 0.105
acc tensor(0.5869, device='cuda:0')
[1,   200] loss: 0.043
acc tensor(0.7606, device='cuda:0')
[2,   100] loss: 0.026
acc tensor(0.8374, device='cuda:0')
[2,   200] loss: 0.024
acc tensor(0.8478, device='cuda:0')
[3,   100] loss: 0.022
acc tensor(0.8573, device='cuda:0')
[3,   200] loss: 0.021
acc tensor(0.8584, device='cuda:0')
[4,   100] loss: 0.019
acc tensor(0.8520, device='cuda:0')
[4,   200] loss: 0.020
acc tensor(0.8709, device='cuda:0')
[5,   100] loss: 0.019
acc tensor(0.8424, device='cuda:0')
[5,   200] loss: 0.019
acc tensor(0.8612, device='cuda:0')
[6,   100] loss: 0.018
acc tensor(0.8592, device='cuda:0')
[6,   200] loss: 0.018
acc tensor(0.8632, device='cuda:0')
[7,   100] loss: 0.017
acc tensor(0.8470, device='cuda:0')
[7,   200] loss: 0.017
acc tensor(0.8817, device='cuda:0')
[8,   100] loss: 0.017
acc tensor(0.8695, device='cuda:0')
[8,   200] loss: 0.017
acc tensor(0.8598, device='cuda:0')
[9,   100] loss: 0.017
acc tensor(0.8728, de

In [12]:
get_accuracy(net,testloader)

tensor(0.8815, device='cuda:0')

In [0]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        #block
        # in 32x32x3
        self.conv_1 = nn.Conv2d(in_channels = 3, out_channels=32,kernel_size=5,stride=1,padding=2) # 32x32x6
        self.batch_norm_1 = nn.BatchNorm2d(32) # same as out_channels before layer 
        self.relu_1 = nn.ReLU()
        self.pool_1 = nn.MaxPool2d(2, 2)

        self.conv_2 = nn.Conv2d(in_channels = 32, out_channels=32,kernel_size=3,stride=1,padding=1)
        self.batch_norm_2 = nn.BatchNorm2d(32) # same as out_channels before layer 
        self.relu_2 = nn.ReLU()
        self.pool_2 = nn.MaxPool2d(2, 2)

        self.conv_3 = nn.Conv2d(in_channels= 32, out_channels=32,kernel_size=3,stride=1,padding=1)
        self.batch_norm_3 = nn.BatchNorm2d(32) # same as out_channels before layer 
        self.relu_3 = nn.ReLU()
        self.pool_3 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(32 * 4 * 4, 10)# 24 chans x 32//(2*2*2)

    def forward(self, x):
        x = self.conv_1(x)
        x = self.batch_norm_1(x)
        x = self.relu_1(x)
        x = self.pool_1(x)


        # skip
        s = x
        x = self.conv_2(x)
        x = self.batch_norm_2(x)
        x = self.relu_2(x)
        x = s + x
        x = self.pool_2(x)

        s= x
        x = self.conv_3(x)
        x = self.batch_norm_3(x)
        x = self.relu_3(x)
        x= s + x
        x = self.pool_3(x)

        x = x.view(-1, 32 * 4 * 4)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc1(x)
        return x


net = MyNet()